<a href="https://colab.research.google.com/github/praveenbenedict/tf2-keras-tutorial/blob/master/13%20-%20Data%20Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-datasets

In [2]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
import tensorflow_datasets as tfds

# To Avoid GPU errors
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

### Load the Cifar 10 dataset
 We'll load the cifar 10 dataset using the tfds module


In [3]:
(ds_train, ds_test), ds_info = tfds.load(
    'cifar10', 
    split = ['train', 'test'],
    shuffle_files = True, 
    as_supervised = True, 
    with_info = True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incompleteTPMHVL/cifar10-train.tfrecord*...:   0%|          …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incompleteTPMHVL/cifar10-test.tfrecord*...:   0%|          |…

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


### Augmenting the dataset
In this section, we'll write helper functions through which each image will be sent through everytime an image is retrieved from the dataset. So, each time we loop through the dataset, the image will be augmented in realtime. 

We'll have to methods, one to simply normalize the image. We won't consider this as augmenting the dataset since it is applied to all the images without much randomness. 

Then, we'll have the augment function which will be used to augment the images and the labels if required. 

In [4]:
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label

In [5]:
def augment(image, label):
    new_height = new_width = 32

    #Resize the image using the new_width and new_height.
    image = tf.image.resize(image, (new_height, new_width))
    
    #Based on a random probability, we convert some images to grayscale.
    if tf.random.uniform((), minval=0, maxval=1) < 0.1:
        image = tf.tile(tf.image.rgb_to_grayscale(image), [1, 1, 3])
    
    #Increase the brightness by a maximum of 10%
    image = tf.image.random_brightness(image, max_delta=0.1)
    #Increase the contrast by a maximum of 10%
    image = tf.image.random_contrast(image, lower=0.1, upper=0.2)
    
    #Randomly flip the image left or right. (Note: This should be done only for some types of datasets)
    image = tf.image.random_flip_left_right(image)
    
    return image, label

In [6]:
ds_train

<PrefetchDataset element_spec=(TensorSpec(shape=(32, 32, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [7]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

Now, let's perform a set of operations to prepare our dataset.

In [8]:
#Pass the normalize_img to the map function of the dataset. This will pass the image and label to the normalize function at run time. 
ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
#Pass the augment function to the map function of the dataset. This will pass the image and label to the normalize function at run time. 
ds_train = ds_train.map(augment, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

In [9]:
ds_test = ds_test.map(normalize_img, num_parallel_calls = AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.prefetch(AUTOTUNE)

In [10]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.Resizing(height=32, width=32),
        layers.experimental.preprocessing.RandomFlip(mode='horizontal'),
        layers.experimental.preprocessing.RandomContrast(factor=0.1)
    ]
)

In [11]:
model = keras.Sequential([
    keras.Input((32, 32, 3)),
    data_augmentation,
    layers.Conv2D(4, 3, padding='same', activation='relu'),
    layers.Conv2D(8, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10)
])

In [12]:
model.compile(
    optimizer=keras.optimizers.Adam(3e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [13]:
model.fit(ds_train, epochs=5, verbose=2)

Epoch 1/5


1563/1563 - 204s - loss: 2.0823 - accuracy: 0.2291 - 204s/epoch - 130ms/step
Epoch 2/5
1563/1563 - 188s - loss: 1.9241 - accuracy: 0.3026 - 188s/epoch - 120ms/step
Epoch 3/5
1563/1563 - 187s - loss: 1.8338 - accuracy: 0.3400 - 187s/epoch - 120ms/step
Epoch 4/5
1563/1563 - 187s - loss: 1.7773 - accuracy: 0.3614 - 187s/epoch - 120ms/step
Epoch 5/5
1563/1563 - 189s - loss: 1.7209 - accuracy: 0.3802 - 189s/epoch - 121ms/step


In [14]:
model.evaluate(ds_test)

313/313 [==============================] - 2s 6ms/step - loss: 4.9324 - accuracy: 0.2623


[4.9323906898498535, 0.2623000144958496]